In [1]:

import numpy as np
from tabulate import tabulate
from numpy.linalg import matrix_rank

def simplex_tableau(variables, basic_vars_index, cost_coefficients, coefficients_matrix, constraints_rhs):
    basic_vars = []
    cost_basic_vars = []
    for index in basic_vars_index:
        basic_vars.append(variables[index])
        cost_basic_vars.append(cost_coefficients[index])
    basic_vars_index = np.transpose([basic_vars_index])
    basic_vars = np.transpose([basic_vars])
    cost_basic_vars = np.transpose([np.array(cost_basic_vars)])
    #Stack coefficients_matrix and cost_coefficients
    simplex_tableau = np.hstack((coefficients_matrix, constraints_rhs)).astype(float)
    simplex_tableau_aux = np.hstack((cost_basic_vars, basic_vars_index)).astype(float)
    return simplex_tableau, simplex_tableau_aux, basic_vars



def display_simplex_tableau(simplex_tableau, simplex_tableau_aux, basic_vars, iteration_count, variables, ratios = False,
                  z = False, objective_function_values = ()):
    print("Simplex Tableau - Iteration " + str(iteration_count))
    simplex_tableau_table = np.hstack((simplex_tableau_aux, basic_vars, simplex_tableau))
    if ratios == False:
        first_row = ["cost_basic_vars", "basic_vars_index", "basic_vars"] + variables.tolist() + ["RHS"]
        objective_function_values = np.hstack((("0","0","zj-cj"),objective_function_values,("0")))
    else:
        first_row = ["cost_basic_vars", "basic_vars_index", "basic_vars"]+variables.tolist()+["RHS"]+["Ratio Test"]
        objective_function_values = np.hstack((("0","0","zj-cj"),objective_function_values,("0","0")))
    if z == False:
        simplex_tableau_table = np.vstack((first_row, simplex_tableau_table))
    else:
        simplex_tableau_table = np.vstack((first_row, simplex_tableau_table, objective_function_values))
    print(tabulate(simplex_tableau_table, headers='firstrow', tablefmt='fancy_grid', 
                   floatfmt=".2f"))




def compute_objective_function(simplex_tableau, simplex_tableau_aux):
    objective_function_values = []
    is_optimal = False
    for index in range(simplex_tableau.shape[1] - 1):
        objective_function_values.append(np.sum(simplex_tableau_aux[:, 0] * simplex_tableau[:, 0 + index])-cost_coefficients[index])
    #Check if optimal
    if all(value <= 0 for value in objective_function_values):
        is_optimal = True 
    return objective_function_values, is_optimal


def identify_pivot_column(simplex_tableau, objective_function_values):
    pivot_column_index = np.argmax(objective_function_values, axis=0)
    pivot_column = simplex_tableau[:, pivot_column_index]
    return pivot_column, pivot_column_index

def perform_ratio_test(simplex_tableau, pivot_column):
    is_unbounded = False          
    ratio_values = np.column_stack([np.array(simplex_tableau[:,-1]) / np.array(pivot_column)])  
    pivot_row_index = np.where(ratio_values > 0, ratio_values, np.inf).argmin()  
    simplex_tableau_display = np.hstack((simplex_tableau, ratio_values))
    if all(value <= 0 for value in ratio_values):
        is_unbounded = True
    return ratio_values, pivot_row_index, simplex_tableau_display, is_unbounded


def perform_row_operations(simplex_tableau, simplex_tableau_aux, basic_vars, variables, pivot_column_index, pivot_row_index, cost_coefficients):    
    pivot_row = simplex_tableau[pivot_row_index, :]
    pivot_value = simplex_tableau[pivot_row_index, pivot_column_index]
    num_rows = simplex_tableau.shape[0]
    num_cols = simplex_tableau.shape[1]
    new_simplex_tableau = simplex_tableau.copy()
    new_simplex_tableau_aux = simplex_tableau_aux.copy()
    for index in range(num_rows):    
        if index != pivot_row_index:
            pivot_value_index = simplex_tableau[index, pivot_column_index]
            ratio = pivot_value_index / pivot_value
            for col_index in range(num_cols):
                new_simplex_tableau[index,col_index] = simplex_tableau[index,col_index] - (ratio * pivot_row[col_index]) 
        else:    
            new_simplex_tableau[index,:] = simplex_tableau[index,:] / pivot_value
    new_simplex_tableau_aux[pivot_row_index,1] = pivot_column_index
    new_simplex_tableau_aux[pivot_row_index, 0] = cost_coefficients[pivot_column_index]
    basic_vars[pivot_row_index] = variables[pivot_column_index]
    return new_simplex_tableau, new_simplex_tableau_aux, basic_vars


def compute_objective_value(simplex_tableau, simplex_tableau_aux, cost_coefficients):
    objective_value = 0
    for index in range(simplex_tableau_aux.shape[0]):
        rhs_value = simplex_tableau[index,-2]
        variable_index = simplex_tableau_aux[index, 1].astype(int)
        cost_coefficient_index = cost_coefficients[variable_index]
        objective_value += (rhs_value * cost_coefficient_index)
    return objective_value


def perform_simplex_iterations(simplex_tableau, simplex_tableau_aux, basic_vars, variables, cost_coefficients):
    iteration_count = 0
    is_unbounded = False
    is_optimal = False
    optimal_value = 0
    while (is_unbounded == False and is_optimal == False):
        
        if iteration_count > 0:
            
            row_operations_info = perform_row_operations(simplex_tableau, simplex_tableau_aux, basic_vars, variables,
                                               pivot_column_index, pivot_row_index, cost_coefficients)
            simplex_tableau = row_operations_info[0]
            simplex_tableau_aux = row_operations_info[1]
            basic_vars = row_operations_info[2]

        
        objective_function_info = compute_objective_function(simplex_tableau, simplex_tableau_aux)
        objective_function_values = objective_function_info[0]
        is_optimal = objective_function_info[1]

        
        pivot_column_info = identify_pivot_column(simplex_tableau, objective_function_values)
        pivot_column = pivot_column_info[0]
        pivot_column_index = pivot_column_info[1]

        
        ratio_test_info = perform_ratio_test(simplex_tableau, pivot_column)
        pivot_row_index = ratio_test_info[1]
        simplex_tableau_with_ratio = ratio_test_info[2]
        if is_optimal == True:
            is_unbounded = False    
        else:
            is_unbounded = ratio_test_info[3]

        
        iteration = iteration_count
        display_simplex_tableau(simplex_tableau_with_ratio, simplex_tableau_aux, basic_vars, iteration_count, variables, 
                  ratios = True, z = True, objective_function_values = objective_function_values)

        iteration_count += 1
    
    
    if is_optimal == True:
        
        optimal_value = compute_objective_value(simplex_tableau_with_ratio, simplex_tableau_aux, cost_coefficients)
        print("Objective Value of the LP")
        print(optimal_value)
        print("At:")
        print(tabulate(np.hstack((basic_vars,np.atleast_2d(simplex_tableau_with_ratio[:,-2]).T)),
                       headers= ("basic_vars","RHS"), tablefmt='fancy_grid', floatfmt=".2f"))
    if is_unbounded == True:
        print("LP is unbounded")
    return(simplex_tableau_with_ratio, simplex_tableau_aux, basic_vars, iteration, optimal_value)





In [2]:
#PROBLEM - MODEL 17 - Farm Fertilizer Problem


two_phase = True

#Original Problem ==================================================
#Vector c - Coefficients of objective function
cost_coefficients_original = np.array([45,42.5,47.5,41.3,13.9,17.8,19.9,12.5,29.9,31.0,24.0,31.2,31.9,35.0,32.5,29.8,9.9,12.3,12.4,11,0,0,0,0,0,0,0,0,0,1,1,1,1,1]) 

#Matrix A - Coefficients of the constraints
coefficients_matrix = np.array([
    [1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,-1,0,0,0,0,0,0,0,0,1],
    [1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
])

# Giving names to variables
variables = np.array(["X11", "X21", "X31","X41","X12","X22","X32","X42","X13","X23","X33","X43","X14","X24","X34","X44","X15","X25,","X35","X45","S1","S2","S3","S4","S5","S6","S7","S8","S9",
                 "A1","A2","A3","A4","A5"])

#Index of the Slack and Artificial Variables that form the Indentity Matrix to form the initial BFS for Phase-1
basic_vars_index = np.array([25,26,27,28,29,30,31,32,33])

# RHS of the constraints
constraints_rhs = np.transpose([np.array([185,50,50,200,185,350,225,195,275])])

#Input artificial variables indexes
artificial_vars_index = np.array([29,30,31,32,33])


cost_coefficients_phase_1 = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1])


In [3]:
#SIMPLEX PHASE 1 =======================================================================
#Iterate Simplex method
simplex_tableau_phase_1, simplex_tableau_aux_phase_1, basic_vars_phase_1, iteration_count, optimal_value_phase_1 = perform_simplex_iterations(
    simplex_tableau, simplex_tableau_aux, basic_vars, variables, cost_coefficients)


NameError: name 'simplex_tableau_aux' is not defined

In [ ]:
#SIMPLEX PHASE 2 =======================================================================
if two_phase == True:
    if optimal_value_phase_1 == 0:
        cost_coefficients = cost_coefficients_original    #utilize objective function original coefficients
        simplex_tableau_phase_1_ex_ratio = simplex_tableau_phase_1[:,:-1]    # remove ratio column
         #remove artificial variables columns
        simplex_tableau_phase_2 = simplex_tableau_phase_1_ex_ratio[:, [index not in artificial_vars_index for index in 
                                             range(simplex_tableau_phase_1_ex_ratio.shape[1])]] 
        simplex_tableau_aux_phase_2 = simplex_tableau_aux_phase_1
        variables_phase_2 = variables[[index not in artificial_vars_index for index in range(variables.shape[0])]]
        basic_vars_phase_2 = basic_vars_phase_1
        for index in range(basic_vars_phase_2.shape[0]):    #get coefficients of the original 
            #objective value for phase 2
            variable_index = int(simplex_tableau_aux_phase_2[index,1])    #indexes of the basic variables
            simplex_tableau_aux_phase_2[index,0] = cost_coefficients[variable_index]    #get original coefficients of basic 
            #variables
        simplex_phase_2 = perform_simplex_iterations(simplex_tableau_phase_2, simplex_tableau_aux_phase_2, basic_vars_phase_2, variables_phase_2, cost_coefficients)
    else:
        print("LP is infeasible")
